## Step 1: Main Dataset

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
# Load datasets
df_raw = pd.read_csv("https://code.s3.yandex.net/datasets/gift.csv")
df_raw_description = pd.read_csv("https://code.s3.yandex.net/python-for-analytics/gift_entry.csv', sep=';")

# Create copies of the datasets to work with
df = df_raw.copy()
df_description = df_raw_description.copy()

# Examine general information about the main dataset
df.info()
df.describe()

In [ ]:
# Look at 10 random rows from the main dataset
df.sample(10)


In [ ]:
# Look at 10 random rows from the supplementary dataset
df_description.sample(10)

After reviewing the data, several conclusions can be drawn

1) Negative values are observed in the customer_id column

2) Negative values are also observed in the quanity column. Quantity also cannot have negative indicators

3) Negative values are also observed in the price column. The price also cannot have negative indicators

4) In the quantity column we observe an abnormally high value (80995). If we compare it with the average or medial value, we have an excess of several hundred times

5) There are no gaps

### Checking for gaps

In [ ]:
# Check for missing values in the main dataset
print("Main dataset - missing values:")
print(df.isnull().sum())

# Check for missing values in the dataset with record descriptions
print("\nDataset with record descriptions - missing values:")
print(df_description.isnull().sum())

There was one gap in the dataset with a description - we delete it

In [ ]:
# Drop rows with missing values in the dataset with record descriptions
df_description = df_description.dropna()
print("\nDataset with record descriptions - missing values removed:")
print(df_description.isnull().sum())

### Checking for duplicates

In [ ]:
# Check for duplicates in the main dataset
print("Main dataset - number of duplicates:")
print(df.duplicated().sum())

# Check for explicit duplicates in df_description
duplicate_count = df_description[df_description['entry'].str.lower().duplicated()].shape[0]
print("\nDataset with record descriptions - number of duplicates:")
print(duplicate_count)

There are obvious duplicates in the main dataset (3573 pieces) in the main dataset and (3 pieces) in the description dataset. We also delete them

In [ ]:
# Remove duplicates in the main dataset
df = df.drop_duplicates()

# Remove duplicates in the dataset with record descriptions

# Create a temporary copy of the 'entry' column converted to lowercase
temp_lower_entry = df_description['entry'].str.lower()

# Identify indices of duplicates
duplicate_indices = temp_lower_entry[temp_lower_entry.duplicated()].index

# Remove duplicates from the main dataframe
df_description_cleaned = df_description.drop(duplicate_indices)

# Check for duplicates after removal
print("\nNumber of duplicates removed from the main dataset:")
print(len(df_raw) - len(df))

# Output number of duplicates
print("\nNumber of duplicates removed from the description dataset:")
print(len(duplicate_indices))

# Calculate the percentage of removed rows and explicit duplicates in the main dataset
percentage_removed = ((len(df_raw) - len(df)) / len(df_raw)) * 100

print("\nPercentage of rows removed from the main dataset:")
print(f"{percentage_removed:.4f}%")

We removed 1% of rows as part of data preprocessing from the main dataset. In the dataset with the description, only 4 lines were removed.

### Checking datasets for matching ID numbers

In [ ]:
# Check the presence of identification numbers in both dataframes
main_ids = set(df['entry_id'])
description_ids = set(df_description['entry_id'])

# Find common identification numbers
common_ids = main_ids.intersection(description_ids)

# Find identification numbers that are only in the main dataset
only_in_main = main_ids - description_ids

# Find identification numbers that are only in the description dataset
only_in_description = description_ids - main_ids

print(f"Total number of identification numbers in the main dataset: {len(main_ids)}")
print(f"Total number of identification numbers in the description dataset: {len(description_ids)}")
print(f"Number of common identification numbers: {len(common_ids)}")
print(f"Number of identification numbers only in the main dataset: {len(only_in_main)}")
print(f"Number of identification numbers only in the description dataset: {len(only_in_description)}")

### Merging two datasets

In [ ]:
# Merge the two datasets into one (using entry_id as the key)
df_full = df.merge(df_description, on='entry_id', how='outer')

# Create a copy of the merged dataset
df_full_raw = df_full.copy()

# View the main information about the merged dataset
df_full.info()

# Count the number of missing values in each column
print('Number of missing values per column:')
print(df_full.isna().sum())


There are Gaps in the entry column (product name). We are going to study them and decide what to do with them.

### Exploring gaps and "strange" values

In [ ]:
# Create a separate dataframe that includes rows with missing 'entry' values
filtered_df = df_full[df_full['entry'].isna()]

# Examine the dataframe with missing values
filtered_df.describe()

# Calculate the proportion of missing values in the entire dataset
share_of_skips = len(filtered_df) / len(df_full)
print(f"Proportion of missing values: {share_of_skips:.4f}%")

We see that all data has the same customer_id (equal to -1) and the price is not indicated (equal to zero everywhere). Since there is little data with gaps (less than 1%), it was decided to delete them.

In [ ]:

# Remove rows with missing 'entry' values
df_full.dropna(subset=['entry'], inplace=True)

In [ ]:
# Check the result:
print('Number of missing values per column:')
print(df_full.isna().sum())
print("Number of rows after removing missing values:")
print(len(df_full))


In [ ]:
df_full.describe()

Let's examine the columns customer_id (there are fields with the value -1), quantity (negative values), price (negative values)

In [ ]:
# Examine rows where 'customer_id' equals -1
df_full.query('customer_id == -1').sample(10)

In [ ]:
# View statistics for rows where 'customer_id' equals -1
df_full.query('customer_id == -1').describe()

It seems that some orders went under customer_id = -1, which either by mistake did not receive “their” identification number or there was some error in the data. In any case, some of them are real orders and I am not going to delete them. But I will separately consider with values ​​quantity < 0

In [ ]:
# Examine rows where 'quantity' has a negative value
df_full_quantity = df_full.query('quantity < 0')
df_full_quantity

In [ ]:
# Additionally, check the descriptions
entry_unique = df_full_quantity['entry'].unique()
print(entry_unique)

In [ ]:

# Ensure that the price is 0 for these rows
df_full_quantity.describe()

Apparently we are talking about a written-off product or various types of errors. The price for all is 0, so we can safely delete these rows.

In [ ]:
# Update the merged dataset to only include rows with positive 'quantity' values
df_full = df_full.query('quantity > 0')
df_full.describe()

In [ ]:
# Examine rows where 'price' has a negative value
df_full_price = df_full.query('price < 0')
df_full_price

We see only 2 rows where we have negative values ​​in the price column. We are talking about some kind of adjustment. The data will not be needed for future analysis. Delete them

In [ ]:
# Update the merged dataset to only include rows with positive 'price' values
df_full = df_full.query('price > 0')
df_full.describe()

In [ ]:
# Check for values written in uppercase
uppercase_entries = df_full[df_full['entry'].str.isupper()]

print(f"\nNumber of uppercase values in the 'entry' column: {len(uppercase_entries)}")
if len(uppercase_entries) > 0:
    print("Uppercase values:")
    print(uppercase_entries)

    # Get unique uppercase values
    unique_uppercase_entries = uppercase_entries['entry'].unique()

    print(f"\nUnique uppercase values in the 'entry' column: {len(unique_uppercase_entries)}")
    print(unique_uppercase_entries)


In [ ]:
# Identify indices of duplicates
deleted_indices_1 = uppercase_entries.index

# Remove duplicates from the main dataframe
df_full = df_full.drop(deleted_indices_1)

print(f"\nNumber of duplicates removed: {len(deleted_indices_1)}")

Values ​​in the entry column that are written in uppercase are not products. We are talking about various kinds of adjustments and additional actions of the online store (postage, management, commission, and so on). Decided to remove them

In [ ]:
# Calculate the percentage of removed data
deleted_data_1 = ((len(df_full_raw) - len(df_full)) / len(df_full_raw)) * 100
print(f"Percentage of removed data: {deleted_data_1:.4f}%")

## Step 2: Pre-processing and start of exploratory analysis

I have already removed negative values ​​(as well as 0) in the price and quantity columns in the previous step. Now let's look at the abnormally high values ​​(outliers) and decide what to do with them

In [ ]:
df_full.describe()

The quantity and price columns contain abnormally high values. Let's build graphs to clearly see emissions

### Checking the quantity column

In [ ]:
# Set the size of the boxplot
plt.figure(figsize=(10, 6))

# Plot a horizontal boxplot
plt.boxplot(x = df_full['quantity'],  # quantity data
            vert=False,     # horizontal boxplot
            showmeans=True, # show mean on the plot
            meanline=True,  # show mean as a line
            patch_artist=True,  # fill the boxplot with color
            # set outliers color to red
            flierprops=dict(markerfacecolor='red')) 

# Add title to the plot
plt.title("Outlier Analysis of Quantity")

# Add label to the X-axis
plt.xlabel("Quantity")
plt.show()


We accurately remove the abnormally high value (80995). Then let's look at the chart again.

In [ ]:
# Remove abnormally high value
df_full = df_full.query('quantity < 80000')

In [ ]:
df_full.describe()

In [ ]:
# Set the size of the boxplot
plt.figure(figsize=(10, 6))

# Plot a horizontal boxplot
plt.boxplot(x = df_full['quantity'],  # quantity data
            vert=False,     # horizontal boxplot
            showmeans=True, # show mean on the plot
            meanline=True,  # show mean as a line
            patch_artist=True,  # fill the boxplot with color
            # set outliers color to red
            flierprops=dict(markerfacecolor='red')) 

# Add title to the plot
plt.title("Outlier Analysis of Quantity")

# Add label to the X-axis
plt.xlabel("Quantity")
plt.show()

We still see high values ​​in the quantity column, but since buyers are not only individuals, but also wholesale stores, we will leave these values, since they may apply specifically to wholesale buyers.

### Cheking the price column

In [ ]:
# Set the size of the boxplot
plt.figure(figsize=(10, 6))

# Plot a horizontal boxplot
plt.boxplot(x = df_full['price'],  # price data
            vert=False,     # horizontal boxplot
            showmeans=True, # show mean on the plot
            meanline=True,  # show mean as a line
            patch_artist=True,  # fill the boxplot with color
            # set outliers color to red
            flierprops=dict(markerfacecolor='red')) 

# Add title to the plot
plt.title("Outlier Analysis of Price")

# Add label to the X-axis
plt.xlabel("Price")
plt.show()

In [ ]:
# Filter rows where price > 10000
price_over_100 = df_full.query('price > 10000')

# Find unique values in the 'entry' column in the filtered dataset
unique_entries = price_over_100['entry'].unique()

# Display unique entries
print(unique_entries)

Nothing abnormally high was recorded. Just expensive goods

In [ ]:
# Calculate the percentage of deleted data during preprocessing
share_of_deleted_data = (1 - (len(df_full) / len(df_full_raw))) * 100
print(f"Percentage of deleted rows: {share_of_deleted_data:.4f}%")

### Exploring Categorical Columns

In [ ]:
# Define a function to analyze columns
def analyze_column(df_full, column_name):
    print(f"\nColumn Analysis: {column_name}")
    print(f"Number of unique values: {df_full[column_name].nunique()}")
    print(f"Number of missing values: {df_full[column_name].isnull().sum()}")
    print(f"Examples of unique values:\n{df_full[column_name].unique()[:10]}")
    if df_full[column_name].dtype == 'object':
        print(f"Number of values with spaces: {df_full[column_name].str.contains(' ', regex=False).sum()}")
    elif df_full[column_name].dtype in ['int64', 'float64']:
        print(f"Minimum value: {df_full[column_name].min()}")
        print(f"Maximum value: {df_full[column_name].max()}")
        print(f"Mean value: {df_full[column_name].mean()}")
    print("\n")

# Analyze columns
analyze_column(df_full, 'order_id')
analyze_column(df_full, 'customer_id')
analyze_column(df_full, 'name_clust')
analyze_column(df_full, 'entry_id')
analyze_column(df_full, 'country_id')

# Create a column 'price_of_order' where the total cost of each item in the dataset is calculated
df_full['price_of_order'] = df_full['price'] * df_full['quantity']


### Let's calculate by month the number of days in which there were no sales. Let's choose a period for analysis

In [ ]:
# Convert date columns to datetime format
df_full['entry_date'] = pd.to_datetime(df_full['entry_date'])

# Create new columns for year, month, day, hour, and day of the week
df_full['year'] = df_full['entry_date'].dt.year
df_full['month'] = df_full['entry_date'].dt.month
df_full['day'] = df_full['entry_date'].dt.day
df_full['hour'] = df_full['entry_date'].dt.hour
df_full['day_of_week'] = df_full['entry_date'].dt.day_name()

# Count the number of orders for each day of the week
orders_by_day = df_full.groupby('day_of_week')['order_id'].nunique().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

# Visualize the results
plt.figure(figsize=(10, 6))
orders_by_day.plot(kind='bar', color='skyblue')

# Add labels and title
plt.xlabel('Day of the Week')
plt.ylabel('Number of Orders')
plt.title('Number of Orders by Day of the Week')

# Add gridlines
plt.grid(axis='y')

# Display the plot
plt.show()

We see that there are no sales on Tuesdays. I take this as an error in the dataset. Most orders are on Friday-Sunday.

In [ ]:
# Group by year and month, and count the number of unique days
group_by_year_month = df_full.groupby(['year', 'month'])['day'].nunique().reset_index()

# Rename the 'day' column to 'unique_days'
group_by_year_month.rename(columns={'day': 'unique_days'}, inplace=True)

# Display the result
print(group_by_year_month)

Так как присутсвует неполный 12-ый месяц 2019 года, то будем изучасть период такой: 1.1.2019 - 30.11.2019

In [ ]:
# Filter data for the period from January 1, 2019 to November 30, 2019
start_date = '2019-01-01'
end_date = '2019-11-30'

df_full = df_full[(df_full['entry_date'] >= start_date) & (df_full['entry_date'] <= end_date)]

## Step 3: Calculation of metrics

In [ ]:
# Group by days and count the number of unique customers and orders
group_by_days_unique = df_full.groupby(['day'])[['customer_id', 'order_id']].nunique().reset_index()

# Rename the 'customer_id' and 'order_id' columns to 'unique_customers' and 'unique_orders'
group_by_days_unique.rename(columns={'customer_id': 'unique_customers', 'order_id': 'unique_orders'}, inplace=True)

# Display the result
group_by_days_unique

### Number of unique customers and orders by day

In [ ]:
# Plotting the line graph
plt.figure(figsize=(10, 6))

# Line for unique customers
plt.plot(group_by_days_unique['days'], group_by_days_unique['unique_customers'], marker='o', label='Unique Customers')

# Line for unique orders
plt.plot(group_by_days_unique['days'], group_by_days_unique['unique_orders'], marker='x', label='Unique Orders')

# Adding labels and title
plt.xlabel('Day of Purchase')
plt.ylabel('Number of Unique Customers and Orders')
plt.title('Number of Unique Customers and Orders in the period 01-2019 - 11-2019')

# Adding grid
plt.grid(True)

# Adding legend
plt.legend()

# Display the plot
plt.show()


The largest number of unique buyers is at the beginning of the month; by the end of the month there is a noticeable decline, but overall the trend is quite flat. The same trend applies to orders (repeats the curve of unique customers)

### Number of unique customers and orders by hour

In [ ]:
# Grouping by hours and counting unique customers and orders
group_by_hours_unique = df_full.groupby(['hours'])[['customer_id', 'order_id']].nunique().reset_index()

# Renaming columns 'customer_id' and 'order_id' to 'unique_customers' and 'unique_orders'
group_by_hours_unique.rename(columns={'customer_id': 'unique_customers', 'order_id': 'unique_orders'}, inplace=True)

# Displaying the grouped data
group_by_hours_unique

In [ ]:
# Plotting the line graph
plt.figure(figsize=(10, 6))

# Line for unique customers
plt.plot(group_by_hours_unique['hours'], group_by_hours_unique['unique_customers'], marker='o', label='Unique Customers')

# Line for unique orders
plt.plot(group_by_hours_unique['hours'], group_by_hours_unique['unique_orders'], marker='x', label='Unique Orders')

# Adding labels and title
plt.xlabel('Hour of Purchase')
plt.ylabel('Number of Unique Customers and Orders')
plt.title('Number of Unique Customers and Orders in the period 01-2019 - 11-2019')

# Adding grid
plt.grid(True)

# Adding legend
plt.legend()

# Displaying the plot
plt.show()

The peak of orders occurs at lunchtime. In the morning (6-8) and evening (18-20) activity is minimal

In [5]:
### Average revenue by month

In [ ]:
# Grouping by months and calculating the average price of order for each month
group_by_avg_price_month = df_full.groupby(['month'])['price_of_order'].mean().reset_index()

# Grouping by months and counting the number of unique customers
group_by_customer_unique = df_full.groupby(['month'])['customer_id'].nunique().reset_index()

# Renaming column 'price_of_order' to 'average_price_of_order'
group_by_avg_price_month.rename(columns={'price_of_order': 'average_price_of_order'}, inplace=True)

# Renaming column 'customer_id' to 'unique_customer'
group_by_customer_unique.rename(columns={'customer_id': 'unique_customer'}, inplace=True)

# Plotting the line graph for average revenue per month
plt.figure(figsize=(10, 6))
plt.plot(group_by_avg_price_month['month'], group_by_avg_price_month['average_price_of_order'], marker='o')

# Adding labels and title
plt.xlabel('Month')
plt.ylabel('Average Revenue')
plt.title('Average Revenue of the Online Store in the period 01-2019 - 11-2019')

# Adding grid
plt.grid(True)

# Displaying the plot
plt.show()


Studying the graph, we can come to the conclusion that there is no seasonality in the sales of the online store. Average revenue is in the range of 1600 - 2100 units of money, where the peak was the month of August.

In [ ]:
# Plotting the line graph for unique customers per month
plt.figure(figsize=(10, 6))
plt.plot(group_by_customer_unique['month'], group_by_customer_unique['unique_customer'], marker='o')

# Adding labels and title
plt.xlabel('Month')
plt.ylabel('Unique Customers Count')
plt.title('Unique Customers of the Online Store in the period 01-2019 - 11-2019')

# Adding grid
plt.grid(True)

# Displaying the plot
plt.show()


The number of unique customers grew throughout 2019, with a peak in November (1,600 unique customers). Again, there is no point in talking about seasonality.

### Sticky Factor for 2nd and 3rd quarters of 2019

In [ ]:
# Filtering data for the 2nd and 3rd quarters of 2019
df_q2 = df_full[(df_full['entry_date'] >= '2019-04-01') & (df_full['entry_date'] <= '2019-06-30')]
df_q3 = df_full[(df_full['entry_date'] >= '2019-07-01') & (df_full['entry_date'] <= '2019-09-30')]

# Finding unique customers in each quarter
unique_customers_q2 = set(df_q2['customer_id'].unique())
unique_customers_q3 = set(df_q3['customer_id'].unique())

# Finding the intersection of customers who made purchases in both quarters
common_customers = unique_customers_q2.intersection(unique_customers_q3)

# Calculating stickiness factor for the 2nd quarter
stickiness_factor_q2 = len(common_customers) / len(unique_customers_q2)

# Calculating stickiness factor for the 3rd quarter
stickiness_factor_q3 = len(common_customers) / len(unique_customers_q3)

print(f'Sticky Factor for the 2nd quarter of 2019: {stickiness_factor_q2:.2f}')
print(f'Sticky Factor for the 3rd quarter of 2019: {stickiness_factor_q3:.2f}')


### Dividing clients into profiles

In [ ]:
# Creating aggregated profile for each customer
customer_profile = df_full.groupby('customer_id').agg(
    num_orders=('order_id', 'nunique'),                  # Number of unique orders
    first_order_date=('entry_date', 'min'),              # First order date
    last_order_date=('entry_date', 'max'),               # Last order date
    total_quantity=('quantity', 'sum'),                  # Total quantity purchased
    total_amount=('price_of_order', 'sum'),              # Total amount spent
    avg_order_price=('price_of_order', 'mean')           # Average order price
).reset_index()

# Calculating additional metrics
customer_profile['avg_quantity_per_order'] = customer_profile['total_quantity'] / customer_profile['num_orders']

# Sorting by number of orders in descending order
customer_profile = customer_profile.sort_values(by='num_orders', ascending=False)

# Displaying a random sample of 10 rows from the customer profile
customer_profile.sample(10)


### Dividing customers into returning and non-returning customers based on the presence of repeat purchases

In [ ]:
# Dividing customers into returning and non-returning
returning_customers = customer_profile[customer_profile['num_orders'] > 1]
non_returning_customers = customer_profile[customer_profile['num_orders'] == 1]

# Calculating average metrics for each group
returning_avg = returning_customers.mean()
non_returning_avg = non_returning_customers.mean()

print("Average metrics for returning customers:")
print(returning_avg[['num_orders', 'total_quantity', 'total_amount', 'avg_order_price', 'avg_quantity_per_order']])

print("\nAverage metrics for non-returning customers:")
print(non_returning_avg[['total_quantity', 'total_amount', 'avg_order_price', 'avg_quantity_per_order']])


Returning customers logically have a much higher average average. What is interesting is that the average order price was almost the same in both groups (3003 units of money for returnable ones versus 2835 units of money for non-refundable ones), as well as the average number of goods in the order (167 units for returnable ones versus 158 units for non-refundable ones). non-refundable). On average, return customers have 6 orders per study period.

## Step 4: RFM customer segmentation

In [ ]:
# Current date (for recency calculation)
current_date = pd.to_datetime('2024-06-06')

# Calculating RFM metrics
rfm_df = df_full.groupby('customer_id').agg({
    'entry_date': lambda x: (current_date - x.max()).days,
    'order_id': 'count',
    'price_of_order': 'sum'
}).reset_index()

rfm_df.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']

rfm_df.head()


### Evaluation of the resulting groups and recommendations for business on interaction with segments

In [ ]:
# Define a function to assign scores based on quantiles
def rfm_score(x, quantiles, reverse=False):
    if reverse:
        return pd.qcut(x, q=4, labels=False, duplicates='drop')
    else:
        return pd.qcut(x, q=4, labels=False, duplicates='drop')

# Calculate quantiles for RFM metrics
quantiles = rfm_df.quantile(q=[0.25, 0.5, 0.75])

# Assign R, F, M scores
rfm_df['R_score'] = rfm_score(rfm_df['Recency'], quantiles['Recency'], reverse=True).astype(int)
rfm_df['F_score'] = rfm_score(rfm_df['Frequency'], quantiles['Frequency']).astype(int)
rfm_df['M_score'] = rfm_score(rfm_df['Monetary'], quantiles['Monetary']).astype(int)

# Calculate RFM segment
rfm_df['RFM_segment'] = rfm_df['R_score'].astype(str) + rfm_df['F_score'].astype(str) + rfm_df['M_score'].astype(str)

# Calculate RFM score
rfm_df['RFM_score'] = rfm_df[['R_score', 'F_score', 'M_score']].sum(axis=1)

rfm_df.head()


In [ ]:
# Segment description
segment_summary = rfm_df.groupby('RFM_segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'customer_id': 'count'
}).reset_index()

segment_summary.columns = ['RFM_segment', 'Avg_Recency', 'Avg_Frequency', 'Avg_Monetary', 'Count_Customers']

segment_summary


In [ ]:
# Visualizing RFM segments
plt.figure(figsize=(10, 6))
plt.bar(segment_summary['RFM_segment'], segment_summary['Count_Customers'], color='skyblue')
plt.xlabel('RFM Segment')
plt.ylabel('Number of Customers')
plt.title('Distribution of Customers across RFM Segments')
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True)) 
plt.show()


Based on the analysis, the following recommendations can be formulated:

1) Most valuable customers (RFM_score 9-12):

-Maintain a high level of service.

-Offer exclusive offers and loyalty programs.

2) Clients on the verge of leaving (RFM_score 1-4):

-Develop retention campaigns.

-Offer discounts and special promotions.

3) Clients with average activity (RFM_score 5-8): -Stimulate repeat purchases.

-Offer personalized recommendations.

## Step 5: Testing statistical hypotheses

### Comparison of the share of returning and non-returning customers for the second and third quarters of 2019

In [ ]:
# Filtering data for the second and third quarters of 2019
second_quarter_2019 = df_full[(df_full['entry_date'] >= '2019-04-01') & (df_full['entry_date'] <= '2019-06-30')]
third_quarter_2019 = df_full[(df_full['entry_date'] >= '2019-07-01') & (df_full['entry_date'] <= '2019-09-30')]

# Defining returning customers (customers who made more than one order)
def returning_customers(df_full):
    return df_full.groupby('customer_id').filter(lambda x: len(x) > 1)['customer_id'].nunique()

# Defining non-returning customers (customers who made only one order)
def non_returning_customers(df_full):
    return df_full.groupby('customer_id').filter(lambda x: len(x) == 1)['customer_id'].nunique()

# Counting the number of returning and non-returning customers in the second and third quarters
returning_2q_2019 = returning_customers(second_quarter_2019)
non_returning_2q_2019 = non_returning_customers(second_quarter_2019)
returning_3q_2019 = returning_customers(third_quarter_2019)
non_returning_3q_2019 = non_returning_customers(third_quarter_2019)

print(f'2Q2019: Returning: {returning_2q_2019}, Non-returning: {non_returning_2q_2019}')
print(f'3Q2019: Returning: {returning_3q_2019}, Non-returning: {non_returning_3q_2019}')


In [ ]:
# Contingency table
contingency_table = [
    [returning_2q_2019, non_returning_2q_2019],
    [returning_3q_2019, non_returning_3q_2019]
]

# Chi-square test
chi2, p, dof, ex = stats.chi2_contingency(contingency_table)

print(f'Chi2: {chi2}, p-value: {p}')


The p-value (0.4756) significantly exceeds the significance level of 0.05, which indicates that there are no statistically significant differences in the shares of returning and non-returning customers between the second and third quarters of 2019.

Based on the chi-square test performed, there is no sufficient reason to reject the null hypothesis. This means that the shares of returning and non-returning customers in the second and third quarters of 2019 are not statistically different. Thus, we can conclude that customer repayment behavior has not changed between these two quarters.

### Comparison of average receipts in countries with country_id equal to 3, 6 and 24

In [ ]:
# Filter data by country_id
countries = [3, 6, 24]
country_data = df_full[df_full['country_id'].isin(countries)]

# Group by country_id and calculate average order price
country_avg_check = country_data.groupby('country_id')['price_of_order'].mean().reset_index()
country_avg_check

In [ ]:
# Applying ANOVA statistical test
country3 = country_data[country_data['country_id'] == 3]['price_of_order']
country6 = country_data[country_data['country_id'] == 6]['price_of_order']
country24 = country_data[country_data['country_id'] == 24]['price_of_order']

f_stat, p_val = stats.f_oneway(country3, country6, country24)

print(f'F-statistic: {f_stat}, p-value: {p_val}')

The p-value (2.2169939757520933e-170) is significantly less than the significance level of 0.05, which indicates that there are statistically significant differences between average receipts in countries with country_id 3, 6 and 24.

Based on the ANOVA test, we can conclude that average checks in countries with country_id 3, 6 and 24 are statistically significantly different. This means that the average purchase amounts differ between these countries, which may indicate differences in purchasing behavior or economic conditions between these countries.

### Own hypothesis - Customers who have made purchases worth more than 1800 units of money have a greater chance of becoming returning customers

In [ ]:
# Filter customers by purchase amount
high_spenders = rfm_df[rfm_df['Monetary'] > 1800]
low_spenders = rfm_df[rfm_df['Monetary'] <= 1800]

# Calculate returning customers
high_returning = high_spenders[high_spenders['Frequency'] > 1].shape[0]
low_returning = low_spenders[low_spenders['Frequency'] > 1].shape[0]

# Calculate total number of customers
high_total = high_spenders.shape[0]
low_total = low_spenders.shape[0]

print(f'Returning customers with spending > 1800 units of money: {high_returning}, Total customers with spending > 1800 units of money: {high_total}')
print(f'Returning customers with spending <= 1800 units of money: {low_returning}, Total customers with spending <= 1800 units of money: {low_total}')


In [ ]:
# Contingency table
count = [high_returning, low_returning]
nobs = [high_total, low_total]

# Proportions test
stat, pval = proportions_ztest(count, nobs)

print(f'Z-statistic: {stat}, p-value: {pval}')

The p-value (1.4632831908230165e-94) is significantly less than the 0.05 significance level, indicating that there are statistically significant differences between the proportions of returning customers among those who spent more than 1800 and those who spent less.

Based on the proportion test conducted, it can be concluded that customers who made purchases of more than 1800 are significantly more likely to become returning customers compared to those who spent less. This means that a customer's total spending affects their likelihood to return and make repeat purchases.

General conclusion:

Based on the analysis of online store data, the following conclusions can be drawn:

1) Trends in customer activity and orders:

There is an increase in the number of unique clients at the beginning of the month, followed by a gradual decrease towards the end of the month. However, the overall trend remains quite stable. The peak of orders occurs at lunchtime, and in the morning and evening activity is minimal.

2) Seasonality in sales:

There is no pronounced seasonality in the sales graph. Average revenue remains relatively stable, with a peak in the month of August.

3) Number of unique clients:

There has been an increase in the number of unique customers throughout 2019, peaking in November. Again, seasonality is not observed.

4) Sticky Factor:

Sticky Factor for the 2nd and 3rd quarters of 2019 is 0.6 and 0.56, respectively, which indicates fairly good customer retention.

5) Analysis of returning customers:

Returning customers have higher averages, such as average order price and average number of items per order, compared to non-returning customers. On average, returning customers had more orders during the study period.